In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [31]:
dataset = pd.read_csv('combined_data.csv')

In [32]:
sentences = dataset['text'].tolist()

In [33]:
labels = dataset['sentiment'].tolist()

In [34]:
training_size = len(sentences) * 0.8
training_size = int(training_size)
training_size

1593

In [49]:
training_sentences = sentences[:training_size]
training_labels = labels[:training_size]

test_sentences = sentences[training_size:]
test_labels = labels[training_size:]

x = np.array(training_sentences)
x.shape

(1593,)

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 1000, oov_token = 'OOV')
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen = 100, padding = 'post', truncating = 'post')

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen = 100, padding = 'post', truncating = 'post')

In [44]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(1000, 16, input_length = 100))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(6, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 16)           16000     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 9606      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [45]:
num_epochs = 20
history = model.fit(training_padded, np.array(training_labels), epochs = 20, validation_data = (test_padded, np.array(test_labels)))

Epoch 1/20
50/50 [==============================] - 3s 69ms/step - loss: 0.6921 - accuracy: 0.5210 - val_loss: 0.6931 - val_accuracy: 0.4110
Epoch 2/20
50/50 [==============================] - 0s 2ms/step - loss: 0.6863 - accuracy: 0.5242 - val_loss: 0.7069 - val_accuracy: 0.4110
Epoch 3/20
50/50 [==============================] - 0s 2ms/step - loss: 0.6718 - accuracy: 0.5581 - val_loss: 0.6876 - val_accuracy: 0.4812
Epoch 4/20
50/50 [==============================] - 0s 2ms/step - loss: 0.6408 - accuracy: 0.6183 - val_loss: 0.6820 - val_accuracy: 0.4662
Epoch 5/20
50/50 [==============================] - 0s 2ms/step - loss: 0.5877 - accuracy: 0.6974 - val_loss: 0.6512 - val_accuracy: 0.5840
Epoch 6/20
50/50 [==============================] - 0s 2ms/step - loss: 0.5230 - accuracy: 0.7847 - val_loss: 0.6294 - val_accuracy: 0.6316
Epoch 7/20
50/50 [==============================] - 0s 2ms/step - loss: 0.4642 - accuracy: 0.8675 - val_loss: 0.6145 - val_accuracy: 0.6692
Epoch 8/20
50/50 [=

In [53]:
pred = model.predict_classes(test_padded)
acc = model.evaluate(test_padded, np.array(test_labels))
proba_rnn = model.predict(test_padded)

13/13 [==============================] - 0s 1ms/step - loss: 0.6487 - accuracy: 0.7644


In [54]:
from sklearn.metrics import confusion_matrix
print("Test loss is {0:.2f} accuracy is {1:.2f}  ".format(acc[0],acc[1]))
print(confusion_matrix(pred, test_labels))

Test loss is 0.65 accuracy is 0.76  
[[168  27]
 [ 67 137]]
